# 作业四：语者辨识
根据不同人物的声音信号的MFCC梅尔倒谱系数特征（训练集+测试集），构建Transformer框架，训练网络进行分类任务。

### 数据集

In [ ]:
# 示例代码给的链接已经失效了，推荐直接去Kaggle下载 https://www.kaggle.com/competitions/ml2022spring-hw4/data
!wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partaa
!wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partab
!wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partac
!wget https://github.com/MachineLearningHW/ML_HW4_Dataset/releases/latest/download/Dataset.tar.gz.partad

!cat Dataset.tar.gz.part* > Dataset.tar.gz

# unzip the file
!tar zxvf Dataset.tar.gz

### 设置随机种子

In [2]:
import numpy as np
import torch
import random

def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(666)

### 数据集
整个数据集由以下成分组成：
- data directory \
|---- metadata.json \ |---- testdata.json \ |---- mapping.json \ |---- uttr-{random string}.pt \

其中前三个json文件包含了各类配置，如metadata.json里面包含了梅尔频谱系数的维度信息，不同id对应的语音信息的文件名（这部分用一个巨大的字典来存储）；各类pt文件则是所有的数据经梅尔频谱系数提取后的文件

In [3]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
 
 
class myDataset(Dataset):
	def __init__(self, data_dir, segment_len=128):
		self.data_dir = data_dir
		self.segment_len = segment_len
	
		# Load the mapping from speaker neme to their corresponding id. 
		mapping_path = Path(data_dir) / "mapping.json"
		mapping = json.load(mapping_path.open())
		self.speaker2id = mapping["speaker2id"]
	
		# Load metadata of training data.
		metadata_path = Path(data_dir) / "metadata.json"
		metadata = json.load(open(metadata_path))["speakers"]
	
		# Get the total number of speaker.
		self.speaker_num = len(metadata.keys())
		self.data = []
		for speaker in metadata.keys():
			for utterances in metadata[speaker]:
				self.data.append([utterances["feature_path"], self.speaker2id[speaker]])
 
	def __len__(self):
		return len(self.data)
 
	def __getitem__(self, index):
		feat_path, speaker = self.data[index]
		# Load preprocessed mel-spectrogram.
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		# Segmemt mel-spectrogram into "segment_len" frames.
		if len(mel) > self.segment_len:
			# Randomly get the starting point of the segment.
			start = random.randint(0, len(mel) - self.segment_len)
			# Get a segment with "segment_len" frames.
			mel = torch.FloatTensor(mel[start:start+self.segment_len])
		else:
			mel = torch.FloatTensor(mel)
		# Turn the speaker id into long for computing loss later.
		speaker = torch.FloatTensor([speaker]).long()
		return mel, speaker
 
	def get_speaker_number(self):
		return self.speaker_num

### 数据加载
这里将所有的训练资料按9:1的比例划分了训练集和验证集

In [4]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
	# Process features within a batch.
	"""Collate a batch of data."""
	mel, speaker = zip(*batch)
	# Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
	mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
	# mel: (batch size, length, 40)
	return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
	"""Generate dataloader"""
	dataset = myDataset(data_dir)
	speaker_num = dataset.get_speaker_number()
	# Split dataset into training dataset and validation dataset
	trainlen = int(0.9 * len(dataset))
	lengths = [trainlen, len(dataset) - trainlen]
	trainset, validset = random_split(dataset, lengths)

	train_loader = DataLoader(
		trainset,
		batch_size=batch_size,
		shuffle=True,
		drop_last=True,
		num_workers=n_workers,
		pin_memory=True,
		collate_fn=collate_batch,
	)
	valid_loader = DataLoader(
		validset,
		batch_size=batch_size,
		num_workers=n_workers,
		drop_last=True,
		pin_memory=True,
		collate_fn=collate_batch,
	)

	return train_loader, valid_loader, speaker_num

### 定义模型
TransformerEncoderLayer：基本Self attention层。参数：
- d_model: 输入的特征数（必需）
- nhead: Multi-head attention的head数（必需）
- dim_feedforward: 前向传播网络模型的维度（默认2048）
- dropout: dropout率（默认0.1）
- activation: 激活层的函数（relu或gelu，默认relu）

TransformerEncoder：几个TransformerEncoderLayer的累加而得。参数：
- encoder_layer: 上面TransformerEncoderLayer()类的一个实例（必需）
- num_layers: 这样的encoder_layer一共有多少层（必需）
- norm: 层归一化选项（可选）

这两个类都是写在torch.nn.modules里面的，可以点进它们的源码查看，里面包括Mutihead-Attention和LayerNorm的计算还有进一步的封装

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Classifier(nn.Module):
	def __init__(self, d_model=80, n_spks=600, dropout=0.1):
		super().__init__()
		# Project the dimension of features from that of input into d_model.
		self.prenet = nn.Linear(40, d_model)
		# TODO:
		#   Change Transformer to Conformer.
		#   https://arxiv.org/abs/2005.08100
		self.encoder_layer = nn.TransformerEncoderLayer(
			d_model=d_model, dim_feedforward=256, nhead=2
		)
		self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=6)

		# Project the the dimension of features from d_model into speaker nums.
		self.pred_layer = nn.Sequential(
			nn.Linear(d_model, d_model),
			nn.ReLU(),
			nn.Linear(d_model, n_spks),
		)

	def forward(self, mels):
		"""
		args:
			mels: (batch size, length, 40)
		return:
			out: (batch size, n_spks)
		"""
		# out: (batch size, length, d_model)
		out = self.prenet(mels)
		# out: (length, batch size, d_model)
		out = out.permute(1, 0, 2)
		# The encoder layer expect features in the shape of (length, batch size, d_model).
		out = self.encoder(out)
		# out: (batch size, length, d_model)
		out = out.transpose(0, 1)
		# mean pooling
		stats = out.mean(dim=1)

		# out: (batch, n_spks)
		out = self.pred_layer(stats)
		return out

### 学习率调整策略
Transformer的Learning rate scheduler和CNN不太一样。研究表明warmup策略对训练transformer模型结构很有帮助。

In [6]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
	optimizer: Optimizer,
	num_warmup_steps: int,
	num_training_steps: int,
	num_cycles: float = 0.5,
	last_epoch: int = -1,
):
	"""
	Create a schedule with a learning rate that decreases following the values of the cosine function between the
	initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
	initial lr set in the optimizer.

	Args:
		optimizer (:class:`~torch.optim.Optimizer`):
		The optimizer for which to schedule the learning rate.
		num_warmup_steps (:obj:`int`):
		The number of steps for the warmup phase.
		num_training_steps (:obj:`int`):
		The total number of training steps.
		num_cycles (:obj:`float`, `optional`, defaults to 0.5):
		The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
		following a half-cosine).
		last_epoch (:obj:`int`, `optional`, defaults to -1):
		The index of the last epoch when resuming training.

	Return:
		:obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
	"""
	def lr_lambda(current_step):
		# Warmup
		if current_step < num_warmup_steps:
			return float(current_step) / float(max(1, num_warmup_steps))
		# decadence
		progress = float(current_step - num_warmup_steps) / float(
			max(1, num_training_steps - num_warmup_steps)
		)
		return max(
			0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
		)

	return LambdaLR(optimizer, lr_lambda, last_epoch)

### 模型正向函数
（例程给的，虽然我感觉这部分完全可以写进main函数里面）

In [7]:
import torch


def model_fn(batch, model, criterion, device):
	"""Forward a batch through the model."""

	mels, labels = batch
	mels = mels.to(device)
	labels = labels.to(device)

	outs = model(mels)

	loss = criterion(outs, labels)

	# Get the speaker id with highest probability.
	preds = outs.argmax(1)
	# Compute accuracy.
	accuracy = torch.mean((preds == labels).float())

	return loss, accuracy

### 验证函数

In [8]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device): 
	"""Validate on validation set."""

	model.eval()
	running_loss = 0.0
	running_accuracy = 0.0
	pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

	for i, batch in enumerate(dataloader):
		with torch.no_grad():
			loss, accuracy = model_fn(batch, model, criterion, device)
			running_loss += loss.item()
			running_accuracy += accuracy.item()

		pbar.update(dataloader.batch_size)
		pbar.set_postfix(
			loss=f"{running_loss / (i+1):.2f}",
			accuracy=f"{running_accuracy / (i+1):.2f}",
		)

	pbar.close()
	model.train()

	return running_accuracy / len(dataloader)

### Main函数

In [10]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset/Dataset",
		"save_path": "model.ckpt",
		"batch_size": 32,
		"n_workers": 8,
		"valid_steps": 2000,
		"warmup_steps": 1000,
		"save_steps": 10000,
		"total_steps": 70000,
	}

	return config


def main(
	data_dir,
	save_path,
	batch_size,
	n_workers,
	valid_steps,
	warmup_steps,
	total_steps,
	save_steps,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
	train_iterator = iter(train_loader)
	print(f"[Info]: Finish loading data!",flush = True)

	model = Classifier(n_spks=speaker_num).to(device)
	criterion = nn.CrossEntropyLoss()
	optimizer = AdamW(model.parameters(), lr=1e-3)
	scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
	print(f"[Info]: Finish creating model!",flush = True)

	best_accuracy = -1.0
	best_state_dict = None

	pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

	for step in range(total_steps):
		# Get data
		try:
			batch = next(train_iterator)
		except StopIteration:
			train_iterator = iter(train_loader)
			batch = next(train_iterator)

		loss, accuracy = model_fn(batch, model, criterion, device)
		batch_loss = loss.item()
		batch_accuracy = accuracy.item()

		# Updata model
		loss.backward()
		optimizer.step()
		scheduler.step()
		optimizer.zero_grad()

		# Log
		pbar.update()
		pbar.set_postfix(
			loss=f"{batch_loss:.2f}",
			accuracy=f"{batch_accuracy:.2f}",
			step=step + 1,
		)

		# Do validation
		if (step + 1) % valid_steps == 0:
			pbar.close()

			valid_accuracy = valid(valid_loader, model, criterion, device)

			# keep the best model
			if valid_accuracy > best_accuracy:
				best_accuracy = valid_accuracy
				best_state_dict = model.state_dict()

			pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

		# Save the best model so far.
		if (step + 1) % save_steps == 0 and best_state_dict is not None:
			torch.save(best_state_dict, save_path)
			pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

	pbar.close()


if __name__ == "__main__":
	main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [01:07<00:00, 29.54 step/s, accuracy=0.06, loss=4.30, step=2000]
Valid: 100% 5664/5667 [00:02<00:00, 2588.62 uttr/s, accuracy=0.10, loss=4.43]
Train: 100% 2000/2000 [01:06<00:00, 30.06 step/s, accuracy=0.28, loss=3.31, step=4000]
Valid: 100% 5664/5667 [00:02<00:00, 2543.60 uttr/s, accuracy=0.24, loss=3.50]
Train: 100% 2000/2000 [01:05<00:00, 30.32 step/s, accuracy=0.22, loss=3.40, step=6000]
Valid: 100% 5664/5667 [00:02<00:00, 2645.88 uttr/s, accuracy=0.32, loss=3.08]
Train: 100% 2000/2000 [01:06<00:00, 29.88 step/s, accuracy=0.28, loss=3.51, step=8000]
Valid: 100% 5664/5667 [00:02<00:00, 2628.87 uttr/s, accuracy=0.39, loss=2.70]
Train: 100% 2000/2000 [01:05<00:00, 30.53 step/s, accuracy=0.38, loss=2.45, step=1e+4]
Valid: 100% 5664/5667 [00:02<00:00, 2591.93 uttr/s, accuracy=0.44, loss=2.48]
Train:   0% 6/2000 [00:00<01:18, 25.53 step/s, accuracy=0.56, loss=2.06, step=1e+4]

Step 10000, best model saved. (accuracy=0.4356)


Train: 100% 2000/2000 [01:06<00:00, 30.10 step/s, accuracy=0.41, loss=2.24, step=12000]
Valid: 100% 5664/5667 [00:02<00:00, 2586.20 uttr/s, accuracy=0.50, loss=2.21]
Train: 100% 2000/2000 [01:06<00:00, 30.08 step/s, accuracy=0.53, loss=1.57, step=14000]
Valid: 100% 5664/5667 [00:02<00:00, 2604.01 uttr/s, accuracy=0.50, loss=2.16]
Train: 100% 2000/2000 [01:07<00:00, 29.82 step/s, accuracy=0.62, loss=1.61, step=16000]
Valid: 100% 5664/5667 [00:02<00:00, 2633.45 uttr/s, accuracy=0.55, loss=1.97]
Train: 100% 2000/2000 [01:06<00:00, 29.99 step/s, accuracy=0.56, loss=1.69, step=18000]
Valid: 100% 5664/5667 [00:02<00:00, 2602.74 uttr/s, accuracy=0.57, loss=1.88]
Train: 100% 2000/2000 [01:06<00:00, 29.88 step/s, accuracy=0.59, loss=1.79, step=2e+4] 
Valid: 100% 5664/5667 [00:02<00:00, 2609.22 uttr/s, accuracy=0.59, loss=1.79]
Train:   0% 6/2000 [00:00<01:17, 25.79 step/s, accuracy=0.44, loss=1.88, step=2e+4]

Step 20000, best model saved. (accuracy=0.5939)


...

Step 50000, best model saved. (accuracy=0.7694)


Train: 100% 2000/2000 [01:06<00:00, 30.27 step/s, accuracy=0.91, loss=0.39, step=52000]
Valid: 100% 5664/5667 [00:02<00:00, 2625.99 uttr/s, accuracy=0.77, loss=1.04]
Train: 100% 2000/2000 [01:06<00:00, 30.27 step/s, accuracy=0.91, loss=0.38, step=54000]
Valid: 100% 5664/5667 [00:02<00:00, 2617.80 uttr/s, accuracy=0.77, loss=0.99]
Train: 100% 2000/2000 [01:06<00:00, 30.14 step/s, accuracy=0.91, loss=0.51, step=56000]
Valid: 100% 5664/5667 [00:02<00:00, 2659.88 uttr/s, accuracy=0.79, loss=1.00]
Train: 100% 2000/2000 [01:06<00:00, 30.05 step/s, accuracy=0.75, loss=1.08, step=58000]
Valid: 100% 5664/5667 [00:02<00:00, 2548.91 uttr/s, accuracy=0.79, loss=0.94]
Train: 100% 2000/2000 [01:06<00:00, 30.08 step/s, accuracy=0.84, loss=0.85, step=6e+4] 
Valid: 100% 5664/5667 [00:02<00:00, 2746.13 uttr/s, accuracy=0.78, loss=0.97]
Train:   0% 5/2000 [00:00<01:17, 25.83 step/s, accuracy=0.97, loss=0.14, step=6e+4]

Step 60000, best model saved. (accuracy=0.7887)


Train: 100% 2000/2000 [01:06<00:00, 30.26 step/s, accuracy=0.91, loss=0.31, step=62000]
Valid: 100% 5664/5667 [00:02<00:00, 2662.56 uttr/s, accuracy=0.79, loss=0.94]
Train: 100% 2000/2000 [01:06<00:00, 29.85 step/s, accuracy=0.97, loss=0.18, step=64000]
Valid: 100% 5664/5667 [00:02<00:00, 2604.13 uttr/s, accuracy=0.80, loss=0.93]
Train: 100% 2000/2000 [01:05<00:00, 30.39 step/s, accuracy=0.94, loss=0.58, step=66000]
Valid: 100% 5664/5667 [00:02<00:00, 2674.16 uttr/s, accuracy=0.80, loss=0.95]
Train: 100% 2000/2000 [01:05<00:00, 30.34 step/s, accuracy=0.78, loss=0.83, step=68000]
Valid: 100% 5664/5667 [00:02<00:00, 2610.06 uttr/s, accuracy=0.79, loss=0.95]
Train: 100% 2000/2000 [01:06<00:00, 30.19 step/s, accuracy=0.81, loss=0.55, step=7e+4] 
Valid: 100% 5664/5667 [00:02<00:00, 2579.05 uttr/s, accuracy=0.79, loss=0.96]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 70000, best model saved. (accuracy=0.7975)


### Inference代码
首先准备数据集

In [11]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
	def __init__(self, data_dir):
		testdata_path = Path(data_dir) / "testdata.json"
		metadata = json.load(testdata_path.open())
		self.data_dir = data_dir
		self.data = metadata["utterances"]

	def __len__(self):
		return len(self.data)

	def __getitem__(self, index):
		utterance = self.data[index]
		feat_path = utterance["feature_path"]
		mel = torch.load(os.path.join(self.data_dir, feat_path))

		return feat_path, mel


def inference_collate_batch(batch):
	"""Collate a batch of data."""
	feat_paths, mels = zip(*batch)

	return feat_paths, torch.stack(mels)

### 做推理并输出提交到Kaggle的结果

In [13]:
import json
import csv
from pathlib import Path
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
	"""arguments"""
	config = {
		"data_dir": "./Dataset/Dataset",
		"model_path": "./model.ckpt",
		"output_path": "./output.csv",
	}

	return config


def main(
	data_dir,
	model_path,
	output_path,
):
	"""Main function."""
	device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	print(f"[Info]: Use {device} now!")

	mapping_path = Path(data_dir) / "mapping.json"
	mapping = json.load(mapping_path.open())

	dataset = InferenceDataset(data_dir)
	dataloader = DataLoader(
		dataset,
		batch_size=1,
		shuffle=False,
		drop_last=False,
		num_workers=8,
		collate_fn=inference_collate_batch,
	)
	print(f"[Info]: Finish loading data!",flush = True)

	speaker_num = len(mapping["id2speaker"])
	model = Classifier(n_spks=speaker_num).to(device)
	model.load_state_dict(torch.load(model_path))
	model.eval()
	print(f"[Info]: Finish creating model!",flush = True)

	results = [["Id", "Category"]]
	for feat_paths, mels in tqdm(dataloader):
		with torch.no_grad():
			mels = mels.to(device)
			outs = model(mels)
			preds = outs.argmax(1).cpu().numpy()
			for feat_path, pred in zip(feat_paths, preds):
				results.append([feat_path, mapping["id2speaker"][str(pred)]])

	with open(output_path, 'w', newline='') as csvfile:
		writer = csv.writer(csvfile)
		writer.writerows(results)


if __name__ == "__main__":
	main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


100%|██████████| 8000/8000 [00:39<00:00, 203.27it/s]
